### Building & Evaluating Complex Agents with Strands and flotorch-eval

In this notebook, we'll go through a complete example of building complex agents easily using strands agents and then evaluate it's performance.
Steps we'll go through:
- **Define specialist agents**: Create individual agents that has specific tasks based on the use case
- **Build orchestrator**: Create a master agent that intelligently delegates tasks to appropriate specialist agents
- **Tracing**: Use OpenTelemetry to capture the agent's step-by-step reasoning and tool calls.
- **Evaluate performance**: Use the captured traces to score the agent's performance against ground truth reference on multiple metrics.

#### Setup and dependencies

Installs flotorch-eval, strands-agents and related tools for creating Python agents and tools that can run and evaluate Python code.

In [1]:
# Install dependencies
!pip install flotorch-eval strands-agents strands-agents-tools strands-agents-builder nest_asyncio ragas uv -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 1.8.3 requires docstring-parser<1.0,>=0.16, but you have docstring-parser 0.15 which is incompatible.
instructor 1.8.3 requires rich<14.0.0,>=13.7.0, but you have rich 14.0.0 which is incompatible.


In [2]:
# Optionally install additional libraries that are needed for the use cases. 
# If you do not have them, LLM will identify missing libraries and will try to install in the Python REPL environment
!pip install yfinance matplotlib -q
!pip install beautifulsoup4 pandas requests -q
!pip install pandas numpy scikit-learn matplotlib seaborn joblib -q

In [3]:
# Import libraries

import asyncio
import pandas as pd
from typing import List
from strands.models.bedrock import BedrockModel
from strands.telemetry.tracer import get_tracer
from strands import Agent, tool
from langchain_aws import ChatBedrockConverse
from ragas.llms import LangchainLLMWrapper

from flotorch_eval.agent_eval.metrics.base import BaseMetric
from flotorch_eval.agent_eval.metrics.langchain_metrics import (
    TrajectoryEvalWithLLMMetric,
)
from flotorch_eval.agent_eval.metrics.ragas_metrics import (
    AgentGoalAccuracyMetric,
    ToolCallAccuracyMetric,
)
from flotorch_eval.agent_eval.metrics.latency_metrics import LatencyMetric
from flotorch_eval.agent_eval.metrics.tool_accuracy import ToolAccuracyMetric
from flotorch_eval.agent_eval.metrics.base import MetricConfig

/home/kiran/Flotorch-fork/flotorch-eval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Setting Up Tracing and Evaluation
To evaluate an agent, we first need to understand what it did. We'll use OpenTelemetry to "trace" the agent's execution. The code below sets up a custom QueueSpanExporter, which captures every step (span) of the agent's process and stores it in a queue for later analysis.

In [4]:
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentel_utils import QueueSpanExporter

# Configure the tracer with both console export and our queue export
# Create our queue exporter
queue_exporter = QueueSpanExporter()

# Get the current tracer provider
provider = TracerProvider()

# Add our queue processor
provider.add_span_processor(BatchSpanProcessor(queue_exporter))
tracer = get_tracer(
    enable_console_export=True  # Keep console export for visibility
)

# Add our queue processor to the existing tracer provider
tracer.tracer_provider.add_span_processor(BatchSpanProcessor(queue_exporter))

In [5]:
# Clear the exporter queue for any residue
queue_exporter.clear()

Finally, this asynchronous function ties everything together. It takes an agent, a prompt, and a list of metrics, then performs the entire run-trace-evaluate-display cycle

In [6]:
from evaluation_utils import get_all_spans, create_trajectory, initialize_evaluator, display_evaluation_results

async def evaluate_agent(metrics: List[BaseMetric], spans: List):
    """
    Runs an agent with a given prompt, captures its trace, evaluates it,
    and displays the results.

    Args:
        agent: The agent to be evaluated.
        prompt: The input prompt for the agent.
        metrics: A list of configured metrics for evaluation.
    """

    # 2. Capture and convert the trace
    if not spans:
        print("\nEvaluation failed: No spans were provided.")
        return

    trajectory = create_trajectory(spans)
    print("\n--- Captured Trajectory ---")
    print(trajectory)

    # 3. Initialize and run the evaluator
    evaluator = initialize_evaluator(metrics)
    print("\n--- Running Evaluation ---")
    results = await evaluator.evaluate(trajectory)

    # 4. Display results
    print("\n--- Evaluation Scores ---")
    display_evaluation_results(results)
    return results

We're setting Claude 3.7 Sonnet as the LLM judge will be used to evaluate the agents

In [7]:
# Initialization LLM judge
bedrock_sonnet = ChatBedrockConverse(
    region_name="us-east-1",
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)
llm_judge = LangchainLLMWrapper(bedrock_sonnet)

#### Use Case 1. Data frame manipulation agent

Let's start with a simple, single-agent use case. This agent will act as a data analyst. Its task is to write and execute a Python script using pandas to perform several common data manipulation tasks based on a single prompt.

The Goal: Evaluate how well a general-purpose agent can handle a multi-step, code-generation task from a single instruction.

Amazon Nova Pro, powered by Amazon Bedrock foundation models, will serve as the model behind this agent.
It will be evaluated using Claude 3.7 Sonnet.


In [8]:
dataframe_agent = Agent(model="us.amazon.nova-pro-v1:0")
dataframe_prompt = """
Write a Python script using the pandas library that performs the following tasks:

- Create a sample DataFrame with the columns: 'Name', 'Age', and 'Salary'.
- Add a new column named 'Bonus' that is 10% of the corresponding 'Salary' value.
- Filter the DataFrame to include only rows where the 'Age' is greater than 30.
- Group the data by age brackets (e.g., 20s, 30s, 40s) and calculate the average 'Salary' and 'Bonus' for each group.

Execute the python script and show the output.

Requirements:
- Include clear inline comments to explain the logic.
- Add a docstring for the function, describing its purpose, parameters, and return value.
- Provide an example of how to use the function.
- List any external libraries that need to be installed with pip (if any).
- Include brief documentation describing how the code works and how to run it.
"""

response = dataframe_agent(dataframe_prompt)
print("\n--- Agent Final Response ---")
print(response.message)

Certainly! Below is the Python script that performs the specified tasks using the pandas library. 

### External Libraries
You need to have `pandas` installed. You can install it using pip if you haven't already:
```sh
pip install pandas
```

### Python Script

```python
import pandas as pd

def process_dataframe():
    """
    Create a sample DataFrame, add a 'Bonus' column, filter by age, and group by age brackets.
    
    Returns:
        pd.DataFrame: A DataFrame grouped by age brackets with average 'Salary' and 'Bonus'.
    """
    
    # Create a sample DataFrame
    data = {
        'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
        'Age': [25, 35, 45, 29, 40],
        'Salary': [50000, 60000, 70000, 55000, 65000]
    }
    df = pd.DataFrame(data)
    
    # Add a new column 'Bonus' that is 10% of the 'Salary'
    df['Bonus'] = df['Salary'] * 0.10
    
    # Filter the DataFrame to include only rows where the 'Age' is greater than 30
    df_filtered = df[df['Age'] > 3

To evaluate the agent, we need a "golden answer" to compare its output against. This REFERENCE_FINAL_ANSWER represents the ideal output.

In [9]:
REFERENCE_FINAL_ANSWER = """
import pandas as pd
import numpy as np

def process_employee_data():
    \"""
    Creates a sample employee DataFrame and performs various data manipulations.
    
    This function:
    1. Creates a sample DataFrame with employee information
    2. Adds a bonus column based on salary
    3. Filters employees by age
    4. Groups data by age brackets and calculates statistics
    
    Returns:
        tuple: Contains the original DataFrame, filtered DataFrame, and grouped statistics
    \"""
    # Create a sample DataFrame with employee data
    data = {
        'Name': ['John', 'Alice', 'Bob', 'Emma', 'David', 'Sarah', 'Michael', 'Linda'],
        'Age': [25, 32, 45, 28, 35, 42, 29, 51],
        'Salary': [50000, 65000, 85000, 52000, 72000, 78000, 60000, 95000]
    }
    
    df = pd.DataFrame(data)
    print("Original DataFrame:")
    print(df)
    print("\\n")
    
    # Add a new column 'Bonus' that is 10% of 'Salary'
    df['Bonus'] = df['Salary'] * 0.1
    print("DataFrame with Bonus column added:")
    print(df)
    print("\\n")
    
    # Filter the DataFrame to include only rows where Age > 30
    filtered_df = df[df['Age'] > 30]
    print("Filtered DataFrame (Age > 30):")
    print(filtered_df)
    print("\\n")
    
    # Create a new column for age brackets
    df['Age_Bracket'] = pd.cut(df['Age'], 
                               bins=[20, 30, 40, 50, 60],
                               labels=['20s', '30s', '40s', '50s'])
    
    # Group by age brackets and calculate average Salary and Bonus
    grouped_df = df.groupby('Age_Bracket').agg({
        'Salary': 'mean',
        'Bonus': 'mean'
    }).reset_index()
    
    print("Average Salary and Bonus by Age Bracket:")
    print(grouped_df)
    
    return df, filtered_df, grouped_df

# Execute the function
if __name__ == "__main__":
    original_df, filtered_df, grouped_df = process_employee_data()
"""


In [10]:
REFERENCE_TRAJECTORY_OUTPUTS = [
    {
        "role": "user",
        "content": """
Write a Python script using the pandas library that performs the following tasks:

- Create a sample DataFrame with the columns: 'Name', 'Age', and 'Salary'.
- Add a new column named 'Bonus' that is 10% of the corresponding 'Salary' value.
- Filter the DataFrame to include only rows where the 'Age' is greater than 30.
- Group the data by age brackets (e.g., 20s, 30s, 40s) and calculate the average 'Salary' and 'Bonus' for each group.

Execute the python script and show the output.

Requirements:
- Include clear inline comments to explain the logic.
- Add a docstring for the function, describing its purpose, parameters, and return value.
- Provide an example of how to use the function.
- List any external libraries that need to be installed with pip (if any).
- Include brief documentation describing how the code works and how to run it.
"""
    },
    {
        "role": "assistant",
        "content": """
Certainly! Here is a complete, executable solution with all requirements met.

import pandas as pd
import numpy as np

def process_employee_data():
    '''
    Creates, processes, and analyzes a sample employee DataFrame.

    This function performs the following steps:
    1. Creates a pandas DataFrame with 'Name', 'Age', and 'Salary'.
    2. Adds a 'Bonus' column calculated as 10% of the 'Salary'.
    3. Filters the DataFrame to show only employees older than 30.
    4. Groups the data into age brackets (20s, 30s, 40s, 50s).
    5. Calculates the average salary and bonus for each age group.
    6. Prints each step's result to the console.

    Returns:
        tuple: A tuple containing four DataFrames:
               (df_original, df_with_bonus, filtered_df, grouped_stats)
    '''
    # 1. Create a sample DataFrame
    data = {
        'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Hank'],
        'Age': [25, 35, 45, 28, 52, 31, 38, 41],
        'Salary': [50000, 65000, 80000, 48000, 95000, 60000, 72000, 78000]
    }
    df = pd.DataFrame(data)
    print("--- Original DataFrame ---")
    print(df, "\n")
    
    # Store a copy for the return value before further modifications
    df_original = df.copy()

    # 2. Add a 'Bonus' column that is 10% of the 'Salary'
    df['Bonus'] = df['Salary'] * 0.1
    print("--- DataFrame with 'Bonus' Column ---")
    print(df, "\n")
    df_with_bonus = df.copy()

    # 3. Filter the DataFrame for Age > 30
    filtered_df = df[df['Age'] > 30]
    print("--- Filtered DataFrame (Age > 30) ---")
    print(filtered_df, "\n")

    # 4. Group data by age brackets
    # Define the bins and labels for the age groups
    bins = [20, 30, 40, 50, 60]
    labels = ['20s', '30s', '40s', '50s']
    df['Age_Bracket'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

    # 5. Calculate the average 'Salary' and 'Bonus' for each group
    grouped_stats = df.groupby('Age_Bracket')[['Salary', 'Bonus']].mean().round(2)
    print("--- Average Salary and Bonus by Age Bracket ---")
    print(grouped_stats, "\n")
    
    return df_original, df_with_bonus, filtered_df, grouped_stats

# Example of how to use the function
if __name__ == "__main__":
    # The function prints the output as it runs
    original, with_bonus, filtered, grouped = process_employee_data()

**Output:**
--- Original DataFrame ---
      Name  Age  Salary
0    Alice   25   50000
1      Bob   35   65000
2  Charlie   45   80000
3    David   28   48000
4      Eve   52   95000
5    Frank   31   60000
6    Grace   38   72000
7     Hank   41   78000

--- DataFrame with 'Bonus' Column ---
      Name  Age  Salary    Bonus
0    Alice   25   50000   5000.0
1      Bob   35   65000   6500.0
2  Charlie   45   80000   8000.0
3    David   28   48000   4800.0
4      Eve   52   95000   9500.0
5    Frank   31   60000   6000.0
6    Grace   38   72000   7200.0
7     Hank   41   78000   7800.0

--- Filtered DataFrame (Age > 30) ---
      Name  Age  Salary    Bonus
1      Bob   35   65000   6500.0
2  Charlie   45   80000   8000.0
4      Eve   52   95000   9500.0
5    Frank   31   60000   6000.0
6    Grace   38   72000   7200.0
7     Hank   41   78000   7800.0

--- Average Salary and Bonus by Age Bracket ---
                 Salary     Bonus
Age_Bracket
20s          49000.00   4900.00
30s          65666.67   6566.67
40s          79000.00   7900.00
50s          95000.00   9500.00

---

**External Libraries Required:**
- pandas

Install with:
pip install pandas

text

---

How the Code Works
Initialization: A dictionary data is created and then converted into a pandas DataFrame.
Vectorized Operation: A new Bonus column is created by multiplying the entire Salary column by 0.1. This is a fast, vectorized operation.
Boolean Filtering: A boolean mask (df['Age'] > 30) is created to select and display only the rows that meet this condition.
Binning: The pd.cut function is used to segment the Age data into discrete intervals (bins), which are then assigned descriptive labels.
Grouping & Aggregation: The groupby() method groups all rows by their Age_Bracket. Then, the average (mean()) of the Salary and Bonus columns is calculated for each group.
How to Run the Script
Save the code above into a file named salary_analyzer.py.
Open your terminal or command prompt.
Make sure you have installed the required libraries (pip install pandas numpy).
Navigate to the directory where you saved the file.
Execute the script using the command: python salary_analyzer.py.
"""
    }
]

Now, we define the metrics for evaluation:

ToolAccuracyMetric: Checks if the agent's tool calls (in this case, code execution) were successful.
AgentGoalAccuracyMetric: Uses our LLM judge to compare the agent's final answer to our reference answer.

In [11]:
# Retrieve all spans from queue exporter
spans = get_all_spans(queue_exporter)

In [12]:
dataframe_metrics = [
    ToolAccuracyMetric(),
    TrajectoryEvalWithLLMMetric(
        llm=bedrock_sonnet,
        config=MetricConfig(metric_params={"reference_outputs": REFERENCE_TRAJECTORY_OUTPUTS})
    ),
    AgentGoalAccuracyMetric(
        llm=llm_judge,
        config=MetricConfig(
            metric_params={"reference_answer": REFERENCE_FINAL_ANSWER}
        )
    ),
    LatencyMetric()
]

async def evaluate_dataframe_agent():
    return await evaluate_agent(dataframe_metrics, spans)

# Execute the evaluation
dataframe_results = asyncio.run(evaluate_dataframe_agent())


--- Captured Trajectory ---
trace_id='e0195a2ffd7c903255a1305871ac6346' messages=[Message(role='user', content="\nWrite a Python script using the pandas library that performs the following tasks:\n\n- Create a sample DataFrame with the columns: 'Name', 'Age', and 'Salary'.\n- Add a new column named 'Bonus' that is 10% of the corresponding 'Salary' value.\n- Filter the DataFrame to include only rows where the 'Age' is greater than 30.\n- Group the data by age brackets (e.g., 20s, 30s, 40s) and calculate the average 'Salary' and 'Bonus' for each group.\n\nExecute the python script and show the output.\n\nRequirements:\n- Include clear inline comments to explain the logic.\n- Add a docstring for the function, describing its purpose, parameters, and return value.\n- Provide an example of how to use the function.\n- List any external libraries that need to be installed with pip (if any).\n- Include brief documentation describing how the code works and how to run it.\n", tool_calls=[], time

Metric,Score,Details
tool_accuracy,1.00,- total_calls: 0 - successful_calls: 0 - failed_calls: 0 - errors: []
trajectory_eval_with_llm,1.00,"Let me analyze the accuracy of the given trajectory compared to the reference trajectory: 1. **Logical progression between steps**: - Both trajectories follow a similar logical progression: create DataFrame → add Bonus column → filter by age → group by age brackets → display results. - The actual trajectory moves directly from defined tasks to implementation, maintaining logical flow throughout. 2. **Clear progression**: - The actual trajectory shows clear progression through all required steps. - It starts with importing pandas, defines the function with a docstring, creates the sample data, adds the Bonus column, filters by age, groups by age brackets, and demonstrates usage. 3. **Efficiency**: - The actual trajectory is reasonably efficient, perhaps even more concise than the reference. - The approach to age bracketing is different (using an apply function instead of pd.cut), but this is still a valid approach. 4. **Semantic equivalence to reference trajectory**: - Both trajectories accomplish the same core tasks. - Both include proper docstrings, create sample data, add a Bonus column as 10% of Salary, filter by age > 30, group by age brackets, and calculate averages. - Key differences: - The actual trajectory uses fewer sample rows (5 vs 8) - The actual trajectory uses a custom function for age brackets rather than pd.cut - The actual trajectory doesn't show intermediate step outputs (just final results) - The structure of documentation at the end is formatted differently - These differences don't change the semantic meaning or functionality. Overall, while there are minor implementation differences between the two trajectories, they accomplish the same goals with the same logical flow. The actual trajectory successfully completes all required tasks with good code quality and documentation. Thus, the score should be: true. -------------------- - has_reference: True - raw_score: True"
agent_goal_accuracy,0.00,
latency_summary,0.00,Total Latency: 22835.01 ms Average Step Latency: 7611.67 ms Latency Breakdown: - Strands Agent: 7612.72 ms - Cycle 4e69a4ef-24a8-4cbc-838e-8ca1fd4ee41e: 7611.78 ms - Model invoke: 7610.51 ms


### Use Case 2: Financial advisor orchestrator

Now for a more complex and realistic scenario. We'll build a hierarchical agent system with a master orchestrator that delegates tasks to a team of specialist agents. This is a powerful pattern for building robust and scalable AI systems.

Define the Specialist Agent Tools
First, we define our three specialists. Each specialist is an Agent wrapped in a `@tool` decorator. This turns the entire agent into a callable tool that our orchestrator can use.

**investment_research_assistant**: Knows all about stocks and ETFs. It has its own tool, get_stock_price, to fetch live data.
**budget_optimizer_assistant**: Expert at creating and managing monthly budgets.
**financial_planner_assistant**: Focuses on long-term strategic goals like retirement and buying a house.

In [13]:
# Clear the exporter queue for any residue
queue_exporter.clear()

Since we have three different agents here, we have to determine which models will be behind those agents.
Here we're going with:

**`Amazon Nova Pro`** as investment researcher

**`Amazon Nova lite`** as budget optimizer assistant 

**`Amazon Nova micro`** as financial planner assistant

Agent performance will be evaluated with **`Claude 3.7 Sonnet`** as judge



In [14]:
# Define the tools necessary for the agents

model_research = BedrockModel(model_id="us.amazon.nova-pro-v1:0")

# Investment Research Assistant and it's tool to get the stock prices
@tool
def investment_research_assistant(query: str) -> str:
    investment_researcher = Agent(
        model=model_research,
        system_prompt="""
        You are a financial research analyst who helps users explore various investment opportunities. 
        You specialize in providing insights into stocks, ETFs, mutual funds, bonds, and other instruments.
        You also highlight key market trends, risk factors, and historical performance.
        Your goal is to equip users with comprehensive, objective information to support their investment decisions.
        """,
    )
    return investment_researcher(query).message

# Budget Optimizer Assistant
model_budget = BedrockModel(model_id="us.amazon.nova-lite-v1:0")

@tool
def budget_optimizer_assistant(query: str) -> str:
    budget_coach = Agent(
        model=model_budget,
        system_prompt="""
        You are a smart budgeting assistant who helps users manage and optimize their monthly expenses.
        You analyze income, spending patterns, and savings goals, and suggest personalized recommendations 
        to cut unnecessary costs and improve savings. Your goal is to help users maintain a healthy financial balance.
        """,
    )
    return budget_coach(query).message

# Financial Planner Assistant
model_planner = BedrockModel(model_id="us.amazon.nova-micro-v1:0")

@tool
def financial_planner_assistant(query: str) -> str:
    financial_planner = Agent(
        model=model_planner,
        system_prompt="""
        You are a certified financial advisor bot who helps users create customized financial plans 
        based on their goals, income, age, and risk tolerance. 
        You guide them through budgeting, saving, debt management, insurance, and retirement planning.
        Your goal is to provide practical, step-by-step advice to help users achieve financial stability and growth.
        """,
    )
    return financial_planner(query).message

#### Define Reference Data for Evaluation
For a multi-step orchestrator, we need more than just a final answer. We need a REFERENCE_TRAJECTORY_OUTPUTS, which defines the ideal path of reasoning and tool calls the agent should take.

In [15]:
REFERENCE_FINAL_ANSWER = """
# Your Comprehensive Financial Plan

Based on the detailed analysis from our financial tools, here is your comprehensive three-part financial plan:

## Part 1: Monthly Budget with Aggressive Loan Repayment

**Monthly Income:** $6,000

**Monthly Expenses:**
- Housing: $1,500
- Utilities: $200
- Groceries: $300
- Transportation: $300
- Food (dining out): $200
- Entertainment: $100
- Insurance: $200
- Personal Care: $150
- Miscellaneous: $150
- **Total Monthly Expenses:** $3,200

**Debt and Savings Allocation:**
- Student Loan Payment: $1,613.75
  - Monthly Interest ($113.75) + Extra Principal Payment ($1,500)
- High-Yield Savings Account: $1,000
- Remaining for Emergency Fund/Investment: $186.25

This aggressive payment plan will help you eliminate your student loan much faster than minimum payments while still building your savings.

## Part 2: Investment Strategy for Long-Term Growth

### ETF Comparison:
Both VOO (currently at $555.14) and QQQ (currently at $535.16) are popular ETFs with different focuses:
- **VOO (Vanguard S&P 500 ETF)** tracks the S&P 500 index, providing broad market exposure
- **QQQ (Invesco QQQ Trust)** tracks the Nasdaq-100 index, focusing more on tech companies

### Recommended Portfolio Allocation for Age 30:

**Stocks (70-80%):**
- VOO (Vanguard S&P 500 ETF): 40%
- QQQ (Invesco QQQ Trust): 30%
- VXUS (Vanguard Total International Stock ETF): 20-30%

**Bonds (20-30%):**
- AGG (iShares Core U.S. Aggregate Bond ETF): 20%
- BNDX (Vanguard Total International Bond ETF): 10%

This balanced portfolio provides a good foundation for long-term growth with appropriate diversification across domestic and international markets.

## Part 3: Long-Term Plan for Home Purchase and Retirement

### Timeline to Home Purchase ($450,000 in 7 years):
1. **Years 1-2:** 
   - Aggressively pay off your $25,000 student loan (approximately 12 months)
   - Build emergency fund of 3-6 months expenses ($7,740-$13,900)

2. **Years 3-7:**
   - Save $90,000 for a 20% down payment (approximately 5 years at $1,250/month)
   - Continue retirement contributions
   - Monitor real estate market trends

### Retirement by Age 55:
1. **Start Now:**
   - Contribute 15% of your income to retirement accounts ($10,800/year)
   - Maximize employer match if available
   - Use tax-advantaged accounts (401(k), IRA)

2. **After Home Purchase:**
   - Allocate former down payment savings toward retirement
   - Increase contributions as income grows
   - Review and rebalance investments annually

3. **Ages 40-55:**
   - Accelerate retirement savings
   - Gradually shift to more conservative investments
   - Consider additional income streams

## Action Steps:

1. **Immediate:** Implement the budget with aggressive loan payments while maintaining savings contributions
2. **Within 1 year:** Eliminate student loan debt
3. **Years 1-2:** Complete emergency fund
4. **Years 2-7:** Focus on house down payment while maintaining retirement contributions
5. **Years 7+:** Redirect house savings to retirement after purchase
6. **Ongoing:** Review and adjust plan annually or with major life changes

This comprehensive plan balances your immediate need to eliminate debt while setting you on a path to both homeownership and early retirement. As your income grows, consider increasing your savings rate to accelerate progress toward your goals.
"""


REFERENCE_TRAJECTORY_OUTPUTS = [
    {
        "role": "user",
        "content": """
        I'm a 30-year-old, earning $6,000 per month after taxes. I have $15,000 in a high-yield savings account and a $25,000 student loan with a 5.5% interest rate.
        I need a comprehensive financial plan with three parts:
        1.  A monthly budget that helps me aggressively pay down my loan while still investing.
        2.  An investment strategy for long-term growth. Please compare the recent performance of the VOO and QQQ ETFs and suggest a sample portfolio for my age.
        3.  A long-term plan outlining the steps to buy a $450,000 house in the next 7 years and retire by age 55.
        Can you help create this for me?
        """,
    },
    {
        "role": "assistant",
        "content": "I'll help you create a comprehensive financial plan by consulting with our specialist tools for each part of your request. Let's address each component systematically.",
        "tool_calls": [
            {
                "function": {
                    "name": "budget_optimizer_assistant",
                    "arguments": '{"query": "I earn $6,000 per month after taxes. I have $15,000 in a high-yield savings account and a $25,000 student loan with a 5.5% interest rate. Create a monthly budget that helps me aggressively pay down my loan while still investing."}'
                }
            }
        ]
    },
    {
        "role": "tool",
        "content": """
        Certainly! Here's a structured monthly budget that helps you aggressively pay down your student loan while still allowing you to invest in your high-yield savings account.

        ### Monthly Income
        - **Total Monthly Income:** $6,000

        ### Monthly Expenses
        1. **Housing (Rent/Mortgage):** $1,500
        2. **Utilities (Electricity, Water, Internet, etc.):** $200
        3. **Groceries:** $300
        4. **Transportation (Car payments, Gas, Public Transport):** $300
        5. **Food (Dining out, Eating at restaurants):** $200
        6. **Entertainment (Subscriptions, Hobbies, etc.):** $100
        7. **Insurance (Health, Car, etc.):** $200
        8. **Personal Care (Clothing, Haircuts, etc.):** $150
        9. **Miscellaneous:** $150

        ### Total Monthly Expenses: $3,200

        ### Remaining Income for Loan and Savings
        - **Remaining Income:** $6,000 - $3,200 = $2,800

        ### Loan and Savings Allocation
        1. **Student Loan Payment (Extra Principal Payment):**
           - **Monthly Interest:** $25,000 * 5.5% / 12 = $113.75
           - **Extra Principal Payment:** $1,500 (This is an aggressive payment to reduce the principal faster)
           - **Total Loan Payment:** $113.75 + $1,500 = $1,613.75

        2. **High-Yield Savings Account:**
           - **Monthly Contribution:** $1,000

        ### Summary of Allocation
        - **Student Loan Payment:** $1,613.75
        - **High-Yield Savings Account:** $1,000
        - **Remaining Balance:** $2,800 - $1,613.75 - $1,000 = $176.25

        By following this budget, you'll be aggressively paying down your student loan while still contributing to your savings.
        """,
        "tool_calls": []
    },
    {
        "role": "assistant",
        "content": "Now I'll analyze investment options for your long-term growth strategy, including a comparison of VOO and QQQ ETFs.",
        "tool_calls": [
            {
                "function": {
                    "name": "investment_research_assistant",
                    "arguments": '{"query": "I\'m a 30-year-old looking for an investment strategy for long-term growth. Please compare the recent performance of the VOO and QQQ ETFs and suggest a sample portfolio for someone my age."}'
                }
            }
        ]
    },
    {
        "role": "tool",
        "content": """
        For someone in their 30s looking for long-term growth, a balanced portfolio with a mix of stocks and bonds is typically recommended. Here's a sample portfolio:

        **Current ETF Prices:**
        - VOO (Vanguard S&P 500 ETF): $555.14
        - QQQ (Invesco QQQ Trust): $535.16

        **Recommended Portfolio:**
        1. **Stocks (70-80%)**:
           - **VOO (Vanguard S&P 500 ETF)**: 40%
           - **QQQ (Invesco QQQ Trust)**: 30%
           - **VXUS (Vanguard Total International Stock ETF)**: 20-30%

        2. **Bonds (20-30%)**:
           - **AGG (iShares Core U.S. Aggregate Bond ETF)**: 20%
           - **BNDX (Vanguard Total International Bond ETF)**: 10%

        This portfolio aims for growth with a moderate level of risk. The inclusion of international stocks and bonds helps diversify the portfolio geographically.
        """,
        "tool_calls": []
    },
    {
        "role": "assistant",
        "content": "Finally, I'll create a comprehensive long-term plan that integrates your budget with your home buying and retirement goals.",
        "tool_calls": [
            {
                "function": {
                    "name": "financial_planner_assistant",
                    "arguments": '{"query": "I\'m 30 years old, earning $6,000 per month after taxes. I have $15,000 in a high-yield savings account and a $25,000 student loan with a 5.5% interest rate. Create a long-term plan outlining the steps to buy a $450,000 house in the next 7 years and retire by age 55."}'
                }
            }
        ]
    },
    {
        "role": "tool",
        "content": """
        Here's a step-by-step long-term financial plan:

        ### Step 1: Pay Off Student Loan
        **Student Loan Repayment Strategy:**
        - **Aggressive Repayment:** Monthly payment: $2,166.67 (Principal + Interest)
        - **Time to pay off:** ~12 months

        ### Step 2: Build Emergency Fund
        **Emergency Fund:** Aim to save 3-6 months of expenses ($7,740 - $13,900)
        - Start with $15,000 in savings, add monthly surplus

        ### Step 3: Save for Down Payment
        **Down Payment:** Target: $90,000 (20% of $450,000)
        - **Savings Rate:** Monthly surplus: $1,253.33
        - **Time to save:** $75,000 / $1,253.33 ≈ 60 months (5 years)

        ### Step 4: Retirement Savings
        **Retirement Savings:** Aim to save 15% of income for retirement (annual: $72,000 * 0.15 = $10,800)
        - **Monthly retirement savings:** $1,080

        ### Timeline:
        **Years 1-2:** Pay off student loan, build emergency fund
        **Years 3-7:** Save down payment, continue retirement investing
        **Years 8-15:** Accelerate retirement savings

        By following this plan, you should be well on your way to buying a house and achieving a comfortable retirement by age 55.
        """,
        "tool_calls": []
    },
    {
        "role": "assistant",
        "content": """# Your Comprehensive Financial Plan

Based on the detailed analysis from our financial tools, here is your comprehensive three-part financial plan:

## Part 1: Monthly Budget with Aggressive Loan Repayment

**Monthly Income:** $6,000

**Monthly Expenses:**
- Housing: $1,500
- Utilities: $200
- Groceries: $300
- Transportation: $300
- Food (dining out): $200
- Entertainment: $100
- Insurance: $200
- Personal Care: $150
- Miscellaneous: $150
- **Total Monthly Expenses:** $3,200

**Debt and Savings Allocation:**
- Student Loan Payment: $1,613.75
  - Monthly Interest ($113.75) + Extra Principal Payment ($1,500)
- High-Yield Savings Account: $1,000
- Remaining for Emergency Fund/Investment: $186.25

This aggressive payment plan will help you eliminate your student loan much faster than minimum payments while still building your savings.

## Part 2: Investment Strategy for Long-Term Growth

### ETF Comparison:
Both VOO (currently at $555.14) and QQQ (currently at $535.16) are popular ETFs with different focuses:
- **VOO (Vanguard S&P 500 ETF)** tracks the S&P 500 index, providing broad market exposure
- **QQQ (Invesco QQQ Trust)** tracks the Nasdaq-100 index, focusing more on tech companies

### Recommended Portfolio Allocation for Age 30:

**Stocks (70-80%):**
- VOO (Vanguard S&P 500 ETF): 40%
- QQQ (Invesco QQQ Trust): 30%
- VXUS (Vanguard Total International Stock ETF): 20-30%

**Bonds (20-30%):**
- AGG (iShares Core U.S. Aggregate Bond ETF): 20%
- BNDX (Vanguard Total International Bond ETF): 10%

This balanced portfolio provides a good foundation for long-term growth with appropriate diversification across domestic and international markets.

## Part 3: Long-Term Plan for Home Purchase and Retirement

### Timeline to Home Purchase ($450,000 in 7 years):
1. **Years 1-2:** 
   - Aggressively pay off your $25,000 student loan (approximately 12 months)
   - Build emergency fund of 3-6 months expenses ($7,740-$13,900)

2. **Years 3-7:**
   - Save $90,000 for a 20% down payment (approximately 5 years at $1,250/month)
   - Continue retirement contributions
   - Monitor real estate market trends

### Retirement by Age 55:
1. **Start Now:**
   - Contribute 15% of your income to retirement accounts ($10,800/year)
   - Maximize employer match if available
   - Use tax-advantaged accounts (401(k), IRA)

2. **After Home Purchase:**
   - Allocate former down payment savings toward retirement
   - Increase contributions as income grows
   - Review and rebalance investments annually

3. **Ages 40-55:**
   - Accelerate retirement savings
   - Gradually shift to more conservative investments
   - Consider additional income streams

## Action Steps:

1. **Immediate:** Implement the budget with aggressive loan payments while maintaining savings contributions
2. **Within 1 year:** Eliminate student loan debt
3. **Years 1-2:** Complete emergency fund
4. **Years 2-7:** Focus on house down payment while maintaining retirement contributions
5. **Years 7+:** Redirect house savings to retirement after purchase
6. **Ongoing:** Review and adjust plan annually or with major life changes

This comprehensive plan balances your immediate need to eliminate debt while setting you on a path to both homeownership and early retirement. As your income grows, consider increasing your savings rate to accelerate progress toward your goals."""
    }
]

#### Build and Evaluate the Orchestrator
Now, we create the orchestrator agent. Its system prompt is crucial—it explicitly tells the agent about its available specialist tools and when to use them.

In [16]:
# Define orchestrator system prompt with clear tool selection guidance

MAIN_SYSTEM_PROMPT = """
        You are a certified financial advisor bot who helps users create customized financial plans 
        based on their goals, income, age, and risk tolerance. 
        You guide them through budgeting, saving, debt management, insurance, and retirement planning.
        Your goal is to provide practical, step-by-step advice to help users achieve financial stability and growth.
        """
financial_orchestrator = Agent(
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[investment_research_assistant, budget_optimizer_assistant, financial_planner_assistant]
)

# Define the complex user query
user_query_financial = """
    I'm 30 years old, earning around $6,000 per month. 
    I have some student loans and moderate savings.
    I want to understand how I can better manage my monthly budget, 
    explore investment options, and build a solid long-term financial plan 
    for buying a house and retiring early. Can you help?
    """

financial_orchestrator(user_query_financial)

I'd be happy to help you create a comprehensive financial plan based on your situation. Let's address your needs across budgeting, investments, and long-term financial planning.

First, I'll help you with budget optimization to manage your monthly finances more effectively:
Tool #1: budget_optimizer_assistant
{
    "name": "Model invoke",
    "context": {
        "trace_id": "0xed85bfa68488d70489fe599a63eecc45",
        "span_id": "0xf8d9fc21f8c4b187",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x68acb34cbb906e2a",
    "start_time": "2025-06-12T13:43:41.635808Z",
    "end_time": "2025-06-12T13:44:46.580921Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "gen_ai.event.start_time": "2025-06-12T13:43:41.635810+00:00",
        "gen_ai.system": "strands-agents",
        "agent.name": "Strands Agent",
        "gen_ai.agent.name": "Strands Agent",
        "gen_ai.prompt": "[{\"role\": \"user\", \"content\": [{\"text\

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "## Summary and Next Steps\n\nBased on the information provided, here's an integrated plan to help you achieve your financial goals:\n\n### Budget Management\n- Follow the 50/30/20 rule (50% for needs, 30% for wants, 20% for savings/debt)\n- Potential monthly savings: $2,700 after optimizing your expenses\n- Consider automating your savings to ensure consistency\n- Review and cancel unused subscriptions\n\n### Debt Management\n- Prioritize paying off high-interest student loans\n- Consider refinancing options for your student loans\n- Use either the snowball method (smallest debts first) or avalanche method (highest interest first)\n\n### Emergency Fund\n- Build an emergency fund of 3-6 months of expenses ($18,000-$36,000)\n- Keep this in a high-yield savings account for liquidity\n\n### Investment Strategy\nFor moderate risk and long-term growth:\n- Maximize 401(k) contributions, especially if emplo

In [17]:
# Retrieve all spans from queue exporter
spans = get_all_spans(queue_exporter)

In [18]:
financial_metrics = [
    ToolCallAccuracyMetric(),
    AgentGoalAccuracyMetric(
        llm=llm_judge,
        config=MetricConfig(metric_params={"reference_answer": REFERENCE_FINAL_ANSWER})
    ),
    TrajectoryEvalWithLLMMetric(
        llm=bedrock_sonnet,
        config=MetricConfig(metric_params={"reference_outputs": REFERENCE_TRAJECTORY_OUTPUTS})
    ),
    LatencyMetric()
]

# Run the orchestrator with the user's query
async def evaluate_financial_agent():
    return await evaluate_agent(financial_metrics, spans)

# Execute the evaluation
financial_results = asyncio.run(evaluate_financial_agent())


--- Captured Trajectory ---
trace_id='ed85bfa68488d70489fe599a63eecc45' messages=[Message(role='user', content="\n    I'm 30 years old, earning around $6,000 per month. \n    I have some student loans and moderate savings.\n    I want to understand how I can better manage my monthly budget, \n    explore investment options, and build a solid long-term financial plan \n    for buying a house and retiring early. Can you help?\n    ", tool_calls=[], timestamp=datetime.datetime(2025, 6, 12, 19, 13, 41, 635808)), Message(role='assistant', content="I'd be happy to help you create a comprehensive financial plan based on your situation. Let's address your needs across budgeting, investments, and long-term financial planning.\n\nFirst, I'll help you with budget optimization to manage your monthly finances more effectively:", tool_calls=[ToolCall(name='budget_optimizer_assistant', arguments={'query': "I'm 30 years old with a monthly income of $6,000. I have student loans and moderate savings. H

Metric,Score,Details
tool_call_accuracy,1.00,- evaluation_type: tool_call_accuracy
agent_goal_accuracy,0.00,
trajectory_eval_with_llm,1.00,"Let me evaluate the accuracy of the AI agent's internal trajectory compared to the reference trajectory: 1. Logical progression between steps: - The actual trajectory shows a clear logical progression, starting with budget optimization, then moving to investment options, and finally creating a long-term financial plan. - This matches the reference trajectory's approach of addressing each component systematically. 2. Clear progression: - The actual trajectory demonstrates clear progression through the financial planning process, with each step building on the previous one. - The agent breaks down complex financial concepts into manageable steps, similar to the reference trajectory. 3. Efficiency: - The actual trajectory is relatively efficient, addressing all the key components of financial planning without unnecessary detours. - While the reference trajectory had more specific details about the user's situation (exact student loan amount and interest rate), the actual trajectory still covers all the essential elements of financial planning. 4. Semantic equivalence to reference trajectory: - Both trajectories cover budgeting, investment options, and long-term financial planning. - Both provide specific advice on debt management, emergency funds, investment diversification, and home buying. - Both conclude with a summary that integrates all the components into a cohesive plan. - The actual trajectory lacks some specific details from the reference (like comparing VOO and QQQ ETFs specifically), but it still provides equivalent financial planning guidance. The actual trajectory successfully addresses all the core components of the reference trajectory, showing logical progression, clear steps, and efficiency. While it doesn't have the exact same level of specificity in some areas (due to less specific information in the user query), it provides semantically equivalent financial planning guidance. Thus, the score should be: true. -------------------- - has_reference: True - raw_score: True"
latency_summary,0.00,Total Latency: 403697.54 ms Average Step Latency: 19223.69 ms Latency Breakdown: - Strands Agent: 113006.77 ms - Cycle 8891003e-30b5-487d-a5ad-cec626fadcb6: 70238.32 ms - Model invoke: 64945.11 ms - Tool: budget_optimizer_assistant: 5289.49 ms - Strands Agent: 5288.14 ms - Cycle 66b195bf-e71c-4ee6-b43e-d59dc9770b4d: 5287.23 ms - Model invoke: 5286.17 ms - Cycle c4a889a5-8fc6-4d1e-8686-652528936fe5: 14986.53 ms - Model invoke: 3870.84 ms - Tool: investment_research_assistant: 11113.47 ms - Strands Agent: 11112.37 ms - Cycle 2e719961-bfaa-4001-8f49-433f24ac8abd: 11111.5 ms - Model invoke: 11110.68 ms - Cycle 6e6d4893-c04f-41fb-9c0a-7c1210c039ca: 10503.38 ms - Model invoke: 5333.25 ms - Tool: financial_planner_assistant: 5168.27 ms - Strands Agent: 5167.13 ms - Cycle 264f6677-ffa9-41eb-b938-47f8a123c76c: 5166.37 ms - Model invoke: 5165.16 ms - Cycle 93d99d70-add0-400f-b9d8-bd49e61b8f2f: 17274.23 ms - Model invoke: 17273.13 ms
